In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

The data is downloaded from the [Lending Club Statistics page](https://www.lendingclub.com/info/download-data.action).

In [3]:
loans_2007 = pd.read_csv('data/LoanStats3a.csv', skiprows=1)

To ensure that code runs faster, the following can be removed from LoanStats3a.csv :
- the first line:
    - because it contains the extraneous text Notes offered by Prospectus [link](https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly
- the desc column:
    - which contains a long text explanation for each loan
- all columns containing more than 50% missing values:
    - which allows us to move faster since we can spend less time trying to fill these values

In [5]:
loans_2007.columns


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=145)

In [6]:
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
loans_2007 = loans_2007.drop(['desc'],axis=1)

In [7]:
loans_2007.to_csv("loans_2007.csv")

In [8]:
loans_2007.shape[1]

53

The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the [data dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097) to become familiar with what each column represents. As we understand each feature, we want to pay attention to any features that:  
- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

Removing following:
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completed funded)

In [9]:
remove_cols= [ "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"]
loans_2007= loans_2007.drop(remove_cols, axis=1)

Removing following:
- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)


In [10]:
remove_columns= ["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"]
loans_2007= loans_2007.drop(remove_columns, axis=1)

Removing columns:
- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

In [11]:
remove_columns= ["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"]
loans_2007= loans_2007.drop(remove_columns, axis=1)

Choosing "loan_status" as the target column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. 

In [12]:
loans_2007["loan_status"].value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

Only the "Fully Paid" and "Charged Off" columns describe the final outcome of the loan so we will pnly use them. Therefore we can treat this as a **_Binary Classification problem_**.

We can see that there is a class imbalance here with around 34k values for the Fully Paid category and only around 56oo for the Charged Off. This risks the model being biased towards the bigger class. However, we will deal with this imbalance later in this project.

In [13]:
loans_2007= loans_2007[(loans_2007["loan_status"]=="Charged Off") | (loans_2007["loan_status"]== "Fully Paid")]

replace_status= {"Fully Paid" : 1, "Charged Off" : 0}
loans_2007["loan_status"]= loans_2007["loan_status"].replace(replace_status)

Now, finally, we will drop all those columns with only 1 unique value in them as they don't provide a lot of information and are no use to the model.

In [14]:
drop_columns=[]
for col in loans_2007.columns:
    non_null = loans_2007[col].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique == 1:
        drop_columns.append(col)
        
loans_2007= loans_2007.drop(drop_columns, axis=1)

print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens', 'hardship_flag', 'disbursement_method']


In [15]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies,debt_settlement_flag
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,...,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Apr-2018,0.0,N
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,car,...,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Oct-2016,0.0,N
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,small_business,...,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,Jun-2017,0.0,N
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,other,...,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0,N
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,1,other,...,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,Apr-2018,0.0,N


In [16]:
loans_2007.to_csv("data/filtered_loans_2007.csv")